In [ ]:
import PIL, io, ipywidgets
import keras
import keras.applications.imagenet_utils
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

inputShape = (299, 299, 3)

network = keras.applications.xception
print("[INFO] loading {}...".format(network.__name__))
model = network.Xception(weights="imagenet")

# Generate GAP model to return both classification and final feature maps layers
GAP_model = keras.models.Model(inputs=model.input,
                  outputs=(model.layers[-3].output, model.layers[-1].output))

In [ ]:
# This is the image to be classified with above network
image_file = '/tmp/workspace/Pictures/fuji_temple.jpg'

print("[INFO] loading and pre-processing image...")

pimage = PIL.Image.open(image_file)
buf = io.BytesIO()
pimage.save(buf, format='png')
img = ipywidgets.Image(value=buf.getvalue())
display(img)

In [ ]:
image = keras.preprocessing.image.img_to_array(pimage.resize(inputShape[:2])).astype('uint8')
image = np.expand_dims(image, axis=0)
image = network.preprocess_input(image)

In [ ]:
print("[INFO] classifying image {} with {}...".format(image_file, network.__name__))
last_conv_output, preds = GAP_model.predict(image)
P = keras.applications.imagenet_utils.decode_predictions(preds)

results=[{'rank': i+1, 'score': prob*100.0, 'class': np.where(preds == prob)[1][0], 'id': imagenetID, 'label': label}
         for (i, (imagenetID, label, prob)) in enumerate(P[0])]

print('-' * 60)
print("{0:>5} |{1:>6}  |{2:>6} | {3:<12}| {4:<32}".format("Rank", "Score", "Class", "ImageNet ID", "Label"))
print('-' * 60)
for r in results:
    print("{rank:>5} |{score:>6.2f}% |{class:>6} | {id:<12}| {label:<48}".format(**r))
print('-' * 60)

In [ ]:
def s_jetmap(s):
    g = s[0]
    if g < 64:  return np.asarray((0, 4*g, 255)).astype('uint8')
    if g < 128: return np.asarray((0, 255, 255+4*(64-g))).astype('uint8')
    if g < 192: return np.asarray((4*(g-128), 255, 0)).astype('uint8')
    return np.asarray((255, 255+4*(192-g), 0)).astype('uint8')

def s_graymap(s):
    return np.asarray((s[0], s[0], s[0])).astype('uint8')

def colormap(heatmap, cmap=s_jetmap):
    return np.apply_along_axis(cmap, -1, np.expand_dims(heatmap, axis=-1)).reshape((heatmap.shape[0],heatmap.shape[1],3))


In [ ]:
maxclass = np.argmax(preds[0])
#maxclass = 832

# Perform the GAP-CAM process
last_conv_output = np.squeeze(last_conv_output)
all_amp_layer_weights = model.layers[-1].get_weights()[0]
amp_layer_weights = all_amp_layer_weights[:, maxclass]

fh, fw, fn = last_conv_output.shape
final_output = np.dot(last_conv_output.reshape((fh*fw, fn)), amp_layer_weights).reshape(fh,fw)
final_output /= np.max(final_output)

final_output *= 255
final_output = np.maximum(final_output, 0.0).astype('uint8')
heatmap = colormap(final_output, s_jetmap)
pfimage = PIL.Image.fromarray(heatmap).resize((pimage.width, pimage.height), resample=PIL.Image.BICUBIC)

poimage = keras.preprocessing.image.img_to_array(pimage)
himage = keras.preprocessing.image.img_to_array(pfimage)

eimage = poimage * himage
eimage /= np.max(eimage)
eimage *= 255

eoimage = PIL.Image.fromarray(eimage.astype('uint8'))
buf = io.BytesIO()
eoimage.save(buf, format='png')
img = ipywidgets.Image(value=buf.getvalue())
display(img)